# Amazon Retail data analysis

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName("amzaon").getOrCreate()
spark

In [38]:
pip install Pyarrow


  Using cached pyarrow-17.0.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   -- ------------------------------------- 1.8/25.1 MB 14.3 MB/s eta 0:00:02
   ----- ---------------------------------- 3.4/25.1 MB 9.1 MB/s eta 0:00:03
   ------- -------------------------------- 4.5/25.1 MB 8.6 MB/s eta 0:00:03
   ------- -------------------------------- 4.5/25.1 MB 8.6 MB/s eta 0:00:03
   ------- -------------------------------- 4.5/25.1 MB 8.6 MB/s eta 0:00:03
   ------- -------------------------------- 4.5/25.1 MB 8.6 MB/s eta 0:00:03
   ------- -------------------------------- 4.7/25.1 MB 3.3 MB/s eta 0:00:07
   --------- ------------------------------ 5.8/25.1 MB 3.4 MB/s eta 0:00:06
   ---------- ----------------------------- 6.6/25.1 MB 3.4 MB/s eta 0:00:06
   ------------ --------------------------- 8.1/25.1 MB 3.8 MB/s eta 0:00:05
   -------------- ------------------------- 8.9/25.1 MB 3.9 MB/s eta 0:00:05
   ----

In [39]:
import pyspark.pandas as ps

C:\Users\girib\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [41]:
df=ps.read_excel("./datasets/Amazon Sale Report edit.xlsx",header=True)
df.show(10)

TypeError: read_excel() got an unexpected keyword argument 'squeeze'

1. Calculate the total quantity (Qty) sold for each Category using groupBy and expr.

In [32]:
df.groupBy(df["Category"]).agg(F.expr("sum(Qty) as Total")).show()

+-------------+-----+
|     Category|Total|
+-------------+-----+
| Ethnic Dress| 1053|
|          Top| 9903|
|          Set|45289|
|        Saree|  152|
|       Bottom|  398|
|Western Dress|13943|
|       Blouse|  863|
|        kurta|45045|
|      Dupatta|    3|
+-------------+-----+



2. Find the total revenue (Amount) for each Sales Channel using groupBy and expr.

In [33]:
df.groupBy(df["Sales Channel "]).agg(F.expr("sum(amount) as Total_revenue")).show()

+--------------+-------------------+
|Sales Channel |      Total_revenue|
+--------------+-------------------+
|    Non-Amazon|               NULL|
|     Amazon.in|7.859267829999994E7|
+--------------+-------------------+



3. Determine the average Amount spent on orders based on the ship-state using groupBy and expr.

In [34]:
s = df.groupBy("ship-state").agg(F.expr("round(avg(amount), 2) as average"))
s = s.orderBy(F.desc("average"))
s.show()

+-----------------+-------+
|       ship-state|average|
+-----------------+-------+
|            bihar| 1432.0|
|           LADAKH| 914.01|
|           Sikkim|  901.0|
|            delhi| 827.68|
|       Chandigarh| 823.15|
|         NAGALAND| 801.29|
|      LAKSHADWEEP| 793.82|
|           SIKKIM| 738.64|
|          Mizoram|  735.0|
|           Punjab| 733.77|
|            Bihar| 733.07|
|            BIHAR| 720.61|
|          MANIPUR| 714.02|
|ARUNACHAL PRADESH| 710.71|
|  JAMMU & KASHMIR| 704.06|
|        RAJASTHAN| 695.62|
| HIMACHAL PRADESH| 695.25|
|           ODISHA| 694.79|
|          HARYANA| 688.18|
|    UTTAR PRADESH|  685.3|
+-----------------+-------+
only showing top 20 rows



4. Group orders by Courier Status and count the number of orders in each status.

In [7]:
df.groupBy(df["Courier Status"]).count().show()

+--------------+------+
|Courier Status| count|
+--------------+------+
|       Shipped|109487|
|          NULL|  6872|
|     Cancelled|  5935|
|     Unshipped|  6681|
+--------------+------+



5.Use groupBy and expr to find the maximum and minimum Amount spent per Category.

In [8]:
df.groupBy(df["Category"]).agg(F.expr("max(Amount) as Max"),F.expr("min(Amount) as min")).show()

+-------------+-------+-----+
|     Category|    Max|  min|
+-------------+-------+-----+
| Ethnic Dress| 1449.0|  0.0|
|          Top| 1797.0|  0.0|
|          Set| 5584.0|  0.0|
|        Saree| 2058.0|  0.0|
|       Bottom|1028.58|  0.0|
|Western Dress| 2860.0|  0.0|
|       Blouse|1266.66|  0.0|
|        kurta| 2796.0|  0.0|
|      Dupatta|  305.0|305.0|
+-------------+-------+-----+



 Find the count of orders per Status and Fulfilment type using groupBy.

In [9]:
df.groupBy(df["Fulfilment"]).count().show()

+----------+-----+
|Fulfilment|count|
+----------+-----+
|  Merchant|39277|
|    Amazon|89698|
+----------+-----+



In [10]:
df.columns

['index',
 'Order ID',
 'Date',
 'Status',
 'Fulfilment',
 'Sales Channel ',
 'ship-service-level',
 'Style',
 'SKU',
 'Category',
 'Size',
 'ASIN',
 'Courier Status',
 'Qty',
 'currency',
 'Amount',
 'ship-city',
 'ship-state',
 'ship-postal-code',
 'ship-country',
 'promotion-ids',
 'B2B',
 'fulfilled-by',
 'Unnamed: 22']

Calculate the sum of Amount for each ship-country using groupBy

In [24]:
df.groupBy("ship-country").agg(F.count("*").alias("count")).show()

+------------+------+
|ship-country| count|
+------------+------+
|        NULL|    33|
|          IN|128942|
+------------+------+



Use groupBy to determine the number of unique SKU items sold per ship-city.

In [27]:
df.groupby("ship-city").agg(F.countDistinct("SKU").alias("Unique")).show()

+--------------------+------+
|           ship-city|Unique|
+--------------------+------+
|           Bangalore|   582|
|             GWALIOR|   128|
|        CHEKONIDHARA|     3|
|         KODUNGALLUR|     7|
|            JHINJHAK|     2|
|NANDIVARAM GUDUVA...|    49|
|        Kudukkimotta|     6|
|   Avinashi, Tirupur|     7|
|           Neelipudi|     1|
|VERAVAL RAJKOT DI...|     3|
|          Naranapura|     1|
|              Kathua|     3|
|             SHIMOGA|     3|
|             SANGRUR|    13|
|        Chandauli UP|     1|
|ERANHOLI, Thalassery|     1|
|              Sanand|     1|
|                TEOK|     2|
|        VADAKKUMMURI|     3|
|          REIS MAGOS|     5|
+--------------------+------+
only showing top 20 rows



9. Find the maximum Amount for each ship-service-level across all States.

In [30]:
df.groupby(["ship-service-level","ship-state"]).agg(F.max("Amount").alias("amount")).show()

+------------------+-----------------+------+
|ship-service-level|       ship-state|amount|
+------------------+-----------------+------+
|         Expedited|ARUNACHAL PRADESH|1477.0|
|         Expedited|            ASSAM|1523.0|
|         Expedited| HIMACHAL PRADESH|1556.0|
|         Expedited|           KERALA|1776.0|
|          Standard|           PUNJAB|5495.0|
|         Expedited|           punjab|1115.0|
|         Expedited|             NULL|1112.0|
|          Standard|       TAMIL NADU|2372.0|
|          Standard|              Goa| 999.0|
|          Standard|              GOA|1695.0|
|          Standard|            Bihar|1186.0|
|         Expedited|          MANIPUR|1523.0|
|          Standard|          HARYANA|2796.0|
|         Expedited|        rajasthan| 885.0|
|         Expedited|        TELANGANA|2664.0|
|         Expedited|        MEGHALAYA|1442.0|
|         Expedited|       Puducherry| 568.0|
|         Expedited|       PUDUCHERRY|1449.0|
|         Expedited|          MIZO

Determine the number of unique ASIN products sold in each Category.